In [ ]:
import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Dense, Dropout, Activation, Flatten, 
    Convolution2D, MaxPooling2D, ZeroPadding2D, 
    GlobalAveragePooling2D, AveragePooling2D, BatchNormalization
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import (
    efficientnet, 
    inception_v3, InceptionV3, 
    inception_resnet_v2, InceptionResNetV2, 
    xception, Xception
)

In [ ]:
import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Dense, Dropout, Activation, Flatten, 
    Convolution2D, MaxPooling2D, ZeroPadding2D, 
    GlobalAveragePooling2D, AveragePooling2D, BatchNormalization
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import (
    efficientnet, 
    inception_v3, InceptionV3, 
    inception_resnet_v2, InceptionResNetV2, 
    xception, Xception
)

In [ ]:
%cd /kaggle/input/food-101/
!head 'food-101/food-101/meta/train.txt'

In [ ]:
train = pd.read_csv('food-101/food-101/meta/train.txt', header = None, names=['path'])
test = pd.read_csv('food-101/food-101/meta/test.txt', header = None, names=['path'])

In [ ]:
def spliter(data, class_or_id='id'):
    if class_or_id.upper() == 'CLASS':
        output = data.split('/')[0]
        
    else:
        output = data.split('/')[-1]
    return output

In [ ]:
train['label'] = train['path'].map(lambda x: spliter(data = x, class_or_id = 'Class')) 
train['idx'] = train['path'].map(lambda x: spliter(x)) 
test['label'] = test['path'].map(lambda x: spliter(x, 'class')) 
test['idx'] = test['path'].map(lambda x: spliter(x)) 

In [ ]:
Classes = train['label'].unique()[:20]
print(Classes)

In [ ]:
Labels = []
for c in Classes:
    Labels.append(c.upper())
Classes = [i.upper() for i in Classes]

def format_classes(label):
    if label.upper() in Classes:
        return label
    else:
        return 'others'

train['label'] = train['label'].map(lambda x: format_classes(x))
test['label'] = test['label'].map(lambda x: format_classes(x))
print(train['label'].unique())

In [ ]:
def Define_Path(path):
    return 'food-101/food-101/images/'+path+'.jpg'
train['path'] = train[['path']].apply(Define_Path, axis=1)
test['path'] = test[['path']].apply(Define_Path, axis=1)

In [ ]:
test_images = plt.imread(test['path'].iloc[50])
test_images = test_images/255.
test_images.shape

In [ ]:
train_images = plt.imread(train['path'].iloc[12])
plt.imshow(train_images)
train_images.shape

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )

test_generator = datagen.flow_from_dataframe(dataframe = test, directory=None, x_col='path', y_col='label',
    weight_col=None, target_size=(256, 256), color_mode='rgb', 
    classes=None, class_mode='categorical', batch_size=32,shuffle=False)

train_generator = datagen.flow_from_dataframe(dataframe = train, directory=None, x_col='path', y_col='label',
    weight_col=None, target_size=(256, 256), color_mode='rgb', 
    classes=None, class_mode='categorical', batch_size=32, shuffle=True)

In [ ]:
# Base model
base_model = Xception(weights='imagenet', include_top=False)
x = base_model.output

# Custom layers
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

predictions = Dense(21, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator,
                    steps_per_epoch = len(train_generator) // 32,
                    epochs=125,
                    verbose=1)

In [ ]:
results = model.evaluate(test_generator, batch_size=32)
print(results)

In [ ]:
%cd /kaggle/working/
model.save('Food101.h5')